<a href="https://colab.research.google.com/github/sdavidr/python-random-quote/blob/master/EjemploVinos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
try: #If running in colab 
    import google.colab
    IN_COLAB = True 
    %tensorflow_version 2.x
except:
    IN_COLAB = False

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
#@title Texto de título predeterminado
import numpy as np
#import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization


def get_train_and_test_splits(train_size1, batch_size1=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
            .map(lambda x, y: (x, tf.cast(y, tf.float32)))
            .prefetch(buffer_size=dataset_size)
            .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset1 = (
        dataset.take(train_size1).shuffle(buffer_size=train_size1).batch(batch_size1)
    )
    test_dataset1 = dataset.skip(train_size1).batch(batch_size1)

    return train_dataset1, test_dataset1


hidden_units = [8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset1, test_dataset1):
  
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset1, epochs=num_epochs, validation_data=test_dataset1)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset1, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset1, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")


# Create model inputs

FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs


"""
## Experiment 1: standard neural network
We create a standard deterministic neural network model as a baseline.
"""


def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="selu")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


"""
Let's split the wine dataset into training and test sets, with 85% and 15% of
the examples, respectively.
"""

dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/wine_quality/white/1.0.0.incompleteHL962N/wine_quality-train.tfrecord


  0%|          | 0/4898 [00:00<?, ? examples/s]

Dataset wine_quality downloaded and prepared to /root/tensorflow_datasets/wine_quality/white/1.0.0. Subsequent calls will reuse this data.


In [4]:


num_epochs = 1000
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
with tf.device('/device:GPU:0'):
  (run_experiment(baseline_model, mse_loss, train_dataset, test_dataset))

"""
We take a sample from the test set use the model to obtain predictions for them.
Note that since the baseline model is deterministic, we get a single a
*point estimate* prediction for each test example, with no information about the
uncertainty of the model nor the prediction.
"""

sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

print(examples)
# print each example


predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")


Start training the model...
Epoch 1/1000
17/17 [==============================] - 4s 58ms/step - loss: 35.5868 - root_mean_squared_error: 5.9655 - val_loss: 54.6848 - val_root_mean_squared_error: 7.3949
Epoch 2/1000
17/17 [==============================] - 0s 11ms/step - loss: 33.3764 - root_mean_squared_error: 5.7772 - val_loss: 52.7519 - val_root_mean_squared_error: 7.2631
Epoch 3/1000
17/17 [==============================] - 0s 10ms/step - loss: 31.4292 - root_mean_squared_error: 5.6062 - val_loss: 48.5628 - val_root_mean_squared_error: 6.9687
Epoch 4/1000
17/17 [==============================] - 0s 10ms/step - loss: 29.3348 - root_mean_squared_error: 5.4162 - val_loss: 43.8931 - val_root_mean_squared_error: 6.6252
Epoch 5/1000
17/17 [==============================] - 0s 10ms/step - loss: 27.1002 - root_mean_squared_error: 5.2058 - val_loss: 38.7701 - val_root_mean_squared_error: 6.2266
Epoch 6/1000
17/17 [==============================] - 0s 10ms/step - loss: 24.7293 - root_mean_sq